# Blue-white veil test



## INPUT: image path, mask path
## OUTPUT: yes, no if the blue-white veil is detected

### description:
1. Load images
2. convert image to HSV color space
3. create mask for blue and combine with orginal mask
4. return ratio of blue-white0veil part/ full size of image range [0,1]
5. ratio >0.1 - yes, else no

### WARNING: do not uplad masks that have blue pen marks around lesion, then the result is not reliable

In [2]:
import cv2
import numpy as np


def calculate_blue_white_ratio(image_path, mask_path):
   # Load the image and mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Check if images were loaded successfully
    if image is None:
        print(f"Error loading image: {image_path}")
        return
    if mask is None:
        print(f"Error loading mask: {mask_path}")
        return

    # Resize the mask to match the dimensions of the image
    if image.shape[:2] != mask.shape[:2]:
        print("Image and mask dimensions do not match. Resizing mask...")
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for blue color in HSV
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # Create a mask for blue color
    blue_mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

    # Combine the blue mask with the original mask
    combined_mask = cv2.bitwise_and(mask, mask, mask=blue_mask)



    # Calculate area of blue-white veil
    blue_white_area = cv2.countNonZero(combined_mask)

    # Calculate area of the lesion (non-zero pixels in the mask)
    lesion_area = cv2.countNonZero(mask)

    # Calculate the ratio of blue-white veil area to lesion area
    ratio = blue_white_area / lesion_area

    if ratio>0.1:
        return "yes"
    else:
        return "no"

# File paths relative to the script location
image_path = 'PAT_63_99_189.png'
mask_path = 'PAT_63_99_189_mask.png'

# Calculate blue-white veil area ratio
blue_white_ratio = calculate_blue_white_ratio(image_path, mask_path)
print(f"Ratio of Blue-White Veil to Lesion: {blue_white_ratio}")


Ratio of Blue-White Veil to Lesion: no


## For all photos in the folder, Output in a CSV file

In [3]:
import os
import csv

# Path to the folder containing images and masks
folder_path = '.'  # images, masks, and CSV file are in the same folder

# Open the CSV file for reading
with open('imageID_sheet.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row

    # Open a new CSV file for writing
    with open('comparison_BW_veil.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        # Iterate over rows in the CSV file
        for row in reader:
            image_name = row[0] + '.png'  # Assuming image names are in the first column
            mask_name = row[0] + '_mask.png'
            image_path = os.path.join(folder_path, image_name)
            mask_path = os.path.join(folder_path, mask_name)

            try:
                # Calculate colorfulness score
                blue_white_ratio = calculate_blue_white_ratio(image_path, mask_path)
                
                # Update the row with the computed score
                row.append(blue_white_ratio)

                # Write the updated row to the output CSV file
                writer.writerow(row)
            except FileNotFoundError:
                print(f"File not found: {image_name} or {mask_name}. Skipping...")

print("Done")


Error loading mask: .\PAT_771_1489_345_mask.png
Error loading mask: .\PAT_153_233_45_mask.png
Error loading mask: .\PAT_246_377_159_mask.png
Error loading mask: .\PAT_356_4511_960_mask.png
Error loading mask: .\PAT_1618_2771_628_mask.png
Done


## ps://www.ncbi.nlm.nih.gov/pmc/articles/PMC3160648/   -Automatic detection of Blue-veil by PubMed central

In [4]:
import cv2
import numpy as np

def calculate_blue_white_presence(image_path, mask_path):
    # Load the image and mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Check if images were loaded successfully
    if image is None:
        print(f"Error loading image: {image_path}")
        return
    if mask is None:
        print(f"Error loading mask: {mask_path}")
        return

    # Resize the mask to match the dimensions of the image
    if image.shape[:2] != mask.shape[:2]:
        print("Image and mask dimensions do not match. Resizing mask...")
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    # Convert image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract color channels
    L, a, b = cv2.split(lab_image)
    R, G, B = cv2.split(image)

    # Calculate features
    F1 = R + L
    F2 = G + L
    F3 = B + L
    F4 = R - L
    F5 = G - L
    F6 = B - L

    # Normalize features
    F7 = F4 / (F4 + F5 + F6)
    F8 = F5 / (F4 + F5 + F6)
    F9 = F6 / (F4 + F5 + F6)
    F10 = R - np.mean(R)
    F11 = G - np.mean(G)
    F12 = B - np.mean(B)
    F13 = F10 / (F10 + F11 + F12)
    F14 = F11 / (F10 + F11 + F12)
    F15 = F12 / (F10 + F11 + F12)

    # Calculate blue-white presence based on conditions
    if F3.mean() <= 0.3:
        return "0"
    elif F10.mean() > -51:
        return "0"
    elif F10.mean() <= -194:
        return "0"
    else:
        return "1"

# File paths relative to the script location
image_path = 'PAT_63_99_189.png'
mask_path = 'PAT_63_99_189_mask.png'

# Calculate blue-white presence based on the conditions
blue_white_presence = calculate_blue_white_presence(image_path, mask_path)
print(f"Blue-White Veil Presence: {blue_white_presence}")


Blue-White Veil Presence: 0


## returned in comparison_BW2_veil.csv

In [5]:
import os
import csv

# Path to the folder containing images and masks
folder_path = '.'  # images, masks, and CSV file are in the same folder

# Open the CSV file for reading
with open('imageID_sheet.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row

    # Open a new CSV file for writing
    with open('comparison_BW2_veil.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        # Iterate over rows in the CSV file
        for row in reader:
            image_name = row[0] + '.png'  # Assuming image names are in the first column
            mask_name = row[0] + '_mask.png'
            image_path = os.path.join(folder_path, image_name)
            mask_path = os.path.join(folder_path, mask_name)

            try:
                # Calculate colorfulness score
                blue_white_presence = calculate_blue_white_presence(image_path, mask_path)
                
                # Update the row with the computed score
                row.append(blue_white_presence)

                # Write the updated row to the output CSV file
                writer.writerow(row)
            except FileNotFoundError:
                print(f"File not found: {image_name} or {mask_name}. Skipping...")

print("Done")


C:\Users\cieci\AppData\Local\Temp\ipykernel_17084\3027802829.py:38: RuntimeWarning: divide by zero encountered in divide
  F7 = F4 / (F4 + F5 + F6)
C:\Users\cieci\AppData\Local\Temp\ipykernel_17084\3027802829.py:39: RuntimeWarning: divide by zero encountered in divide
  F8 = F5 / (F4 + F5 + F6)
C:\Users\cieci\AppData\Local\Temp\ipykernel_17084\3027802829.py:40: RuntimeWarning: divide by zero encountered in divide
  F9 = F6 / (F4 + F5 + F6)
C:\Users\cieci\AppData\Local\Temp\ipykernel_17084\3027802829.py:38: RuntimeWarning: invalid value encountered in divide
  F7 = F4 / (F4 + F5 + F6)
C:\Users\cieci\AppData\Local\Temp\ipykernel_17084\3027802829.py:40: RuntimeWarning: invalid value encountered in divide
  F9 = F6 / (F4 + F5 + F6)


Error loading mask: .\PAT_771_1489_345_mask.png


C:\Users\cieci\AppData\Local\Temp\ipykernel_17084\3027802829.py:39: RuntimeWarning: invalid value encountered in divide
  F8 = F5 / (F4 + F5 + F6)


Error loading mask: .\PAT_153_233_45_mask.png
Error loading mask: .\PAT_246_377_159_mask.png
Error loading mask: .\PAT_356_4511_960_mask.png
Error loading mask: .\PAT_1618_2771_628_mask.png
Done
